### Script purpose: Ming office title coding

1. General principles:
    - A comprehensive ontological structure of office title includes four parts: `Classification + Administrative Unit (optional) + Function (optional) + Title`
    - Each part corresponds to a table.
    - Separate `coding_value` and `raw_value`.
        - `raw_value`: the string appeared in original book text.
        - `coding_value`: the revised string that can be successfully coded.

2. Notes:
    - `Office title by LENGTH` table merges CBDB Ming office title with UCI table. Duplicates in CBDB table are removed in this table, i.e., this is the clean table we are going to use.

### TODO:
- [ ] English words;
- [ ] Forward slash.

In [1]:
% matplotlib inline
import sqlite3
import pandas as pd
import networkx as nx
import xlrd
import matplotlib.pyplot as plt
import math
import warnings
from tqdm import tqdm
import re
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

### `c_office_chn` from UCI.

In [2]:
df_uci_office_ming=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSCmhbCk1B-9jjINMhy_VwikM6_Sn7bjdO7b_vaZJkVcYCCYlWVlhYVCFtAs0fPX-UEO62GWxaX1qAS/pub?gid=630627340&single=true&output=tsv',
                                    sep='\t')
df_uci_office_ming=df_uci_office_ming[['c_office_id（Dictionary Ser#)','Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn']].rename(columns={'c_office_id（Dictionary Ser#)':'c_office_id'})
df_uci_office_ming['c_office_chn']=[s.replace('/', '') for s in df_uci_office_ming['c_office_chn']]
df_uci_office_ming.sample(3)

,c_office_id,Institution 1,Institution 2,Institution 3,c_office_chn
2068,2610,地方官署類 Regional and Local Governance,京府門 Superior Prefectural Governance,應天府 Yingtian Superior Prefecture,巡檢司巡檢
3344,71364,皇族宮廷類 Imperial Family and Royal Court,宗室門 The Imperial Clan,王府長史司 Princely Establishment,右長吏
2402,71676,地方軍事與治安機構類 Regional and Local Military Units,(行)都指揮使司門 (Auxiliary) Regional and Local Milit...,衛指揮使司 Guard Military Commands,左屯衛指揮使


In [3]:
df_uci_office_ming['inst_1_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 1']]
df_uci_office_ming['inst_2_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 2']]
df_uci_office_ming['inst_3_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 3']]
df_uci_office_ming['uci_value']=df_uci_office_ming['inst_1_chn']+df_uci_office_ming['inst_2_chn']+df_uci_office_ming['inst_3_chn']+df_uci_office_ming['c_office_chn']
df_uci_office_ming['c_office_id']=pd.to_numeric(df_uci_office_ming['c_office_id'], errors='coerce')
df_uci_office_ming.drop(['inst_1_chn', 'inst_2_chn', 'inst_3_chn', 'Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn'], axis=1, inplace=True)

In [4]:
df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()]

,c_office_id,uci_value
1130,71508.0,中央輔佐官署類秘書門翰林院直文淵閣侍講學士
1195,71503.0,中央輔佐官署類考官門會試官知貢舉官
1219,72165.0,中央輔佐官署類考官門鄉試官順天同考官
1282,NaN,京衛京營與中央軍事官署類京營門京營京營總兵官
2314,71504.0,地方官署類省官門行中書省理問所知事
2718,71274.0,地方軍事與治安機構類招討經略安撫使門宣撫司宣撫司經歷
2821,NaN,文武散階勛爵類勛爵門伯平涼伯
2842,NaN,文武散階勛爵類勛爵門伯新城伯
2862,NaN,文武散階勛爵類勛爵門伯永定伯
2882,NaN,文武散階勛爵類勛爵門伯鎮遠伯


In [5]:
df_uci_office_ming.drop(df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()].index, inplace=True)
df_uci_office_ming.set_index('c_office_id', inplace=True)
df_uci_office_ming.sample(3)

,uci_value
c_office_id,
1122.0,中央中樞官署類六部門禮部膳部郎中
71074.0,地方軍事與治安機構類地區軍官門西路安邊參將
72107.0,皇族宮廷類宗室門藩王永王


### `c_office_chn` from CBDB uncleaned, and merge with UCI.

In [6]:
conn = sqlite3.connect('../../SQL/sqlite_20180302.db')
df_cbdb_office_ming=pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn)[pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn).c_dy==19].set_index('c_office_id')
df_cbdb_office_ming.sample(3)

,tts_sysno,c_dy,c_office_pinyin,c_office_chn,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old
c_office_id,,,,,,,,,,,,,,,,
70895,15632.0,19,tai pu si qing,太僕寺卿,tai pu qing,太僕卿,Chief Minister of the Court of the Imperial Stud,None,NaN,None,None,Chief Minister of the Court of the Imperial Stud,None,None,None,0.0
71320,16057.0,19,yan zheng,鹽政,None,None,Salt Administration,None,NaN,None,None,Salt Administration,None,None,None,0.0
71044,15781.0,19,wen xuan si lang zhong,文選司郎中,wen xuan lang zhong,文選郎中,Director of the Bureau of Appointments of the ...,None,NaN,None,None,Director of the Bureau of Appointments of the ...,None,None,None,0.0


In [7]:
for index in tqdm(df_uci_office_ming.index):
    if index in df_cbdb_office_ming.index:
        df_uci_office_ming.loc[index, 'cbdb_value']=df_cbdb_office_ming.loc[index, 'c_office_chn']
        df_uci_office_ming.loc[index, 'tts_sysno']=df_cbdb_office_ming.loc[index, 'tts_sysno']
        df_uci_office_ming.loc[index, 'c_office_pinyin']=df_cbdb_office_ming.loc[index, 'c_office_pinyin']
        df_uci_office_ming.loc[index, 'c_office_pinyin_alt']=df_cbdb_office_ming.loc[index, 'c_office_pinyin_alt']
        df_uci_office_ming.loc[index, 'c_office_chn_alt']=df_cbdb_office_ming.loc[index, 'c_office_chn_alt']
        df_uci_office_ming.loc[index, 'c_office_trans']=df_cbdb_office_ming.loc[index, 'c_office_trans']
        df_uci_office_ming.loc[index, 'c_office_trans_alt']=df_cbdb_office_ming.loc[index, 'c_office_trans_alt']
        df_uci_office_ming.loc[index, 'c_source']=df_cbdb_office_ming.loc[index, 'c_source']
        df_uci_office_ming.loc[index, 'c_pages']=df_cbdb_office_ming.loc[index, 'c_pages']
        df_uci_office_ming.loc[index, 'c_notes']=df_cbdb_office_ming.loc[index, 'c_notes']
        df_uci_office_ming.loc[index, 'c_category_1']=df_cbdb_office_ming.loc[index, 'c_category_1']
        df_uci_office_ming.loc[index, 'c_category_2']=df_cbdb_office_ming.loc[index, 'c_category_2']
        df_uci_office_ming.loc[index, 'c_category_3']=df_cbdb_office_ming.loc[index, 'c_category_3']
        df_uci_office_ming.loc[index, 'c_category_4']=df_cbdb_office_ming.loc[index, 'c_category_4']
        df_uci_office_ming.loc[index, 'c_office_id_old']=df_cbdb_office_ming.loc[index, 'c_office_id_old']
df_uci_office_ming.loc[index, 'c_dy']=19

100%|██████████| 4304/4304 [00:46<00:00, 93.21it/s]


In [8]:
df_office_ming_merged=df_uci_office_ming
df_office_ming_merged.sample(3)

,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy
c_office_id,,,,,,,,,,,,,,,,,
70829.0,中央輔佐官署類秘書門典籍實錄修纂官世宗實錄副總裁,世宗實錄副總裁,15566.0,shi zong shi lu fu zong cai,None,None,Vice Director-general,NaN,NaN,NaN,NaN,[Not Yet Translated],NaN,NaN,NaN,0.0,NaN
71238.0,中央中樞官署類六部門吏部行在吏部右侍郎,行在吏部右侍郎,15975.0,xing zai li bu you shi lang,None,None,Auxiliary Right Vice Minister of Personnel,NaN,NaN,NaN,NaN,Auxiliary Right Vice Minister of Personnel,NaN,NaN,NaN,0.0,NaN
1944.0,京衛京營與中央軍事官署類大都督府門左軍都督府右斷事,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Coding `c_office_chn`.

### TODO: - Done.
    - [×] Subtract titles from right.
    - [×] Add appointment type.
    - [×] Use online revised CLS table.

In [9]:
df_adm=pd.read_csv('../data_output/C_OT_ADM.tsv', sep='\t').set_index('c_ot_adm_id')
df_cls=pd.read_csv('../data_output/C_OT_CLS.tsv', sep='\t').set_index('c_ot_cls_id')
df_tit=pd.read_csv('../data_output/C_OT_TIT.tsv', sep='\t').set_index('c_ot_tit_id')
df_func=pd.read_csv('../data_output/C_OT_FUNC.tsv', sep='\t').set_index('c_ot_func_id')
df_app_ty=pd.read_csv('../data_output/APPOINTMENT_TYPE_CODES.tsv', sep='\t').set_index('c_appt_type_code')

In [10]:
df_tit.sample(3)

,c_ot_tit_chinm,value_to_run,c_ot_tit_desc,c_ot_tit_start,c_ot_tit_end,length
c_ot_tit_id,,,,,,
1736,理刑行人,1.0,NaN,NaN,NaN,4
1711,鳴贊,1.0,NaN,NaN,NaN,2
1005,選侍,1.0,NaN,NaN,NaN,2


In [11]:
df_office_ming_merged['c_ot_coding']=df_office_ming_merged['uci_value']

In [12]:
re.findall(r'(.+)提舉', '中央中樞官署類六部門戶部寶鈔提舉司提舉')

['中央中樞官署類六部門戶部寶鈔提舉司']

In [13]:
# Replace titles (only one title in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    ming_ot = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
    ming_ot_done=[]
    for tit_index in df_tit.index:
        tit=df_tit.loc[tit_index, 'c_ot_tit_chinm']
        if ming_ot.endswith(tit) and ming_ot not in ming_ot_done:
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_tit_chinm']=tit
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=re.findall(r'(.+)'+tit, ming_ot)[0]+'T'+str(tit_index)
            ming_ot_done.append(ming_ot)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [03:17<00:00, 21.79it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm
c_office_id,,,,,,,,,,,,,,,,,,,
2726.0,地方官署類府官門府官庫副使,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,地方官署類府官門府官庫T1047,副使
1309.0,中央中樞官署類六部門工部總部郎中,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中央中樞官署類六部門工部總部T1326,郎中
70248.0,中央中樞官署類六部門督理西苑農事,督理西苑農事,14985.0,du li xi yuan nong shi,None,None,Supervisory Manager of Agriculture in the West...,NaN,NaN,NaN,NaN,Supervisory Manager of Agriculture in the West...,NaN,NaN,NaN,0.0,NaN,中央中樞官署類六部門T1661,督理西苑農事


In [14]:
# Replace Classifications (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    cls_list=[]
    for cls_index in df_cls.index:
        cls=df_cls.loc[cls_index, 'c_ot_cls_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if cls in c_ot_coding:
            cls_list.append(cls)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(cls, 'C'+str(cls_index))
    if cls_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_cls_chinm']='#'.join(cls_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [00:35<00:00, 122.27it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,
1004.0,中央中樞官署類六部門戶部廣積庫大使,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C11C45戶部廣積庫T1243,大使,中央中樞官署類#六部門
2151.0,京衛京營與中央軍事官署類京衛門親軍都尉府儀鸞司副使,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C0C43親軍都尉府儀鸞司T1047,副使,京衛京營與中央軍事官署類#京衛門
1957.0,京衛京營與中央軍事官署類大都督府門右軍都督府提控案牘,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C0C114右軍都督府T43,提控案牘,京衛京營與中央軍事官署類#大都督府門


In [15]:
# Replace admin units (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    adm_list=[]
    for adm_index in df_adm.index:
        adm=df_adm.loc[adm_index, 'c_ot_adm_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if adm in c_ot_coding:
            adm_list.append(adm)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(adm, 'A'+str(adm_index))
    if adm_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_adm_chinm']='#'.join(adm_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [02:07<00:00, 33.82it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
70352.0,中央中樞官署類六部門工部左侍郎,工部左侍郎,15089.0,gong bu zuo shi lang,None,None,Left Vice Minister of Works,NaN,NaN,NaN,...,Left Vice Minister of Works,NaN,NaN,NaN,0.0,NaN,C11C45A760T104,左侍郎,中央中樞官署類#六部門,工部
70897.0,中央輔佐官署類寺監門太僕寺主簿廳主簿,太僕寺主簿,15634.0,tai pu si zhu bu,None,None,Recorder of the Court of the Imperial Stud,NaN,NaN,NaN,...,Recorder of the Court of the Imperial Stud,NaN,NaN,NaN,0.0,NaN,C8C44A569A544T1575,主簿,中央輔佐官署類#寺監門,太僕寺#主簿廳
72768.0,京衛京營與中央軍事官署類京衛門錦衣衛東廠理刑,理刑,17496.0,li xing,None,None,None,None,NaN,NaN,...,None,NaN,NaN,NaN,0.0,NaN,C0C43A1026A919T1039,理刑,京衛京營與中央軍事官署類#京衛門,錦衣衛#東廠


In [16]:
# Replace functional units (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    func_list=[]
    for func_index in df_func.index:
        func=df_func.loc[func_index, 'c_ot_func_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if func in c_ot_coding:
            func_list.append(func)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(func, 'F'+str(func_index))
    if func_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_func_chinm']='#'.join(func_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [00:16<00:00, 260.53it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm,c_ot_func_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
1598.0,中央輔佐官署類寺監門太僕寺牧監令,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C8C44A569T180,牧監令,中央輔佐官署類#寺監門,太僕寺,NaN
599.0,皇族宮廷類宦官門皇陵署（洪武舊制）Office署令,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C23C47A1072（C66）OfficeT1142,署令,皇族宮廷類#洪武舊制#宦官門,皇陵署,NaN
207.0,皇族宮廷類宦官門司禮監/禮儀房管事,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C23C47A248/A480T941,管事,皇族宮廷類#宦官門,司禮監#禮儀房,NaN


In [17]:
# Replace appointment type.
for ming_ot_index in tqdm(df_office_ming_merged.index):
    app_ty_list=[]
    for app_ty_index in df_app_ty.index:
        app_ty=df_app_ty.loc[app_ty_index, 'c_appt_type_desc_chn']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if app_ty in c_ot_coding:
            app_ty_list.append(app_ty)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(app_ty, 'P'+str(app_ty_index))
    if app_ty_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_app_chinm']='#'.join(app_ty_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [00:17<00:00, 245.96it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm,c_ot_func_chinm,c_ot_app_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
3062.0,地方軍事與治安機構類番夷都指揮使司門番夷衞指揮使司小旗,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,C1C5番夷衞指揮使司T1807,小旗,地方軍事與治安機構類#番夷都指揮使司門,NaN,NaN,NaN
70564.0,京衛京營與中央軍事官署類京營門團營將軍總兵官,將軍總兵官,15301.0,jiang jun zong bing guan,None,None,General-Regional Commander,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,C0C40A1192將軍T258,總兵官,京衛京營與中央軍事官署類#京營門,團營,NaN,NaN
544.0,皇族宮廷類宦官門外差宦官御茶房近侍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,C23C47C67A354T1350,近侍,皇族宮廷類#外差宦官#宦官門,御茶房,NaN,NaN


In [18]:
for index in tqdm(df_office_ming_merged.index):
    c_ot_coding=df_office_ming_merged.loc[index, 'c_ot_coding']
    if re.sub(r'A|C|T|F|P|\d', '', string=c_ot_coding)!='':
        df_office_ming_merged.loc[index, 'pass']='F'
    else:
        df_office_ming_merged.loc[index, 'pass']='T'

100%|██████████| 4304/4304 [00:07<00:00, 605.85it/s]


In [19]:
df_office_ming_merged_coded=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwXjRmlMR9w2ZV2tcenPSz9UgE7WAgeumGxxCJlceQOZRQFgm6_mgMCAlC_GzM0yxxNsDOlU1-5aH-/pub?output=xlsx',
                                          sheetname='merged_tbl_coding'
                                         )
df_office_ming_merged_coded.set_index('c_office_id', inplace=True)
df_office_ming_merged_coded.sample(3)

,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_office_id_old,c_dy,c_ot_coding,type,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm,c_ot_func_chinm,c_ot_app_chinm,pass
c_office_id,,,,,,,,,,,,,,,,,,,,,
71519.0,中央輔佐官署類寺監門中都國子監／學助教,中都國子助教,16256.0,zhong du guo zi zhu jiao,NaN,NaN,Instructor of the Directorate of Education of the,NaN,NaN,NaN,...,0.0,NaN,C8C44中都A567／學T981,W,助教,中央輔佐官署類#寺監門,國子監,NaN,NaN,F
983.0,中央中樞官署類六部門戶部北平清吏司員外郎,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,C11C45A694A83T632,NaN,員外郎,中央中樞官署類#六部門,北平清吏司#戶部,NaN,NaN,T
2779.0,地方官署類縣官門縣官陰陽學訓術,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,C17C49C74陰陽學T1239,NaN,訓術,地方官署類#縣官門#縣官,NaN,NaN,NaN,F


In [20]:
for c_office_id in df_office_ming_merged.index:
    df_office_ming_merged.loc[c_office_id, 'type']=df_office_ming_merged_coded.loc[c_office_id, 'type']

In [21]:
df_office_ming_merged.to_excel('../data_output/ming_office_title_merged_coding.xlsx', encoding='utf8')